In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/product-catalogue/data/processed/public/task_1_query-product_ranking/product_catalogue-v0.2.csv
/kaggle/input/distilbertlm/checkpoint-309354/config.json
/kaggle/input/distilbertlm/checkpoint-309354/trainer_state.json
/kaggle/input/distilbertlm/checkpoint-309354/training_args.bin
/kaggle/input/distilbertlm/checkpoint-309354/tokenizer.json
/kaggle/input/distilbertlm/checkpoint-309354/tokenizer_config.json
/kaggle/input/distilbertlm/checkpoint-309354/pytorch_model.bin
/kaggle/input/distilbertlm/checkpoint-309354/scaler.pt
/kaggle/input/distilbertlm/checkpoint-309354/scheduler.pt
/kaggle/input/distilbertlm/checkpoint-309354/special_tokens_map.json
/kaggle/input/distilbertlm/checkpoint-309354/optimizer.pt
/kaggle/input/distilbertlm/checkpoint-309354/vocab.txt
/kaggle/input/distilbertlm/checkpoint-309354/rng_state.pth
/kaggle/input/distilbertlm/checkpoint-154677/config.json
/kaggle/input/distilbertlm/checkpoint-154677/trainer_state.json
/kaggle/input/distilbertlm/checkpoint-154

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)

In [3]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/84.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━ 41.0/84.0 kB 1.8 MB/s eta 0:00:01


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━ 81.9/84.0 kB 1.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 850.0 kB/s eta 0:00:00


  Preparing metadata (setup.py) ... -

 done


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/101.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.9 MB/s eta 0:00:00


...

 | done


  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.1-py3-none-any.whl size=125774 sha256=8019923b34df3095fa82e64f16a75b9dd44ecb0531d1d7b3829cda45b2dc0fae
  Stored in directory: /root/.cache/pip/wheels/58/27/2f/708b4f002c226e57b6243769da345c650633175c7634f93365
Successfully built sentence-transformers


  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.7.0
    Uninstalling huggingface-hub-0.7.0:


      Successfully uninstalled huggingface-hub-0.7.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cached-path 1.1.3 requires huggingface-hub<0.8.0,>=0.0.12, but you have huggingface-hub 0.8.1 which is incompatible.


In [4]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime

In [5]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

In [6]:
model_name = '../input/distilbertlm/checkpoint-309354'
word_embedding_model = models.Transformer(model_name, max_seq_length=128)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at ../input/distilbertlm/checkpoint-309354 were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
prod_cat = pd.read_csv('../input/product-catalogue/data/processed/public/task_1_query-product_ranking/product_catalogue-v0.2.csv')

In [8]:
prod_cat.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box",NaN,"144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpening\nStrong medium-soft lead produces long-lasting, smooth, readable strokes\nRounded hexagonal shape with satin-smooth finish for a secure, comfortable grip\nSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes",Amazon Basics,Yellow,us
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack","<p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developing new exciting products for our consumers.</p> <p><strong>FOCUSED ON OUR MISSION</strong></p><p>Our currency is ideas. We thrive on imagination, passion and leadership. We have great products and will to continue to rise with our customer expectations.</p> <p><strong>SUCCESS BASED ON SATISFACTION</strong></p><p>Each and every product we send out, we expect our 100% customer satisfaction. While our supplies are sourced from all across the world, our success stems from individual consumer fulfillment. We create products that people want to recommend to others.</p>","&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each pack comes with 12 un-sharpened pencils. Unlike most others out there, these allow kids the simple joy of sharpening a brand new pencil for the very 1st time!\n&#11088; CLASSIC BARREL. Made from premium-quality wood pencil is bonded with a break-resistant number two core for smooth writing as easily erased. Hexagonal shape prevents rolling and comfortable to hold.\n&#11088; QUALITY. Pencils are #2 HB for general writing, drawing and sketching. Pencils glide smoothly over paper and is highly break resistant. Pencils include a smudge-resistant eraser top for convenience.\n&#11088; LATEX-FREE ERASER. The smudge-less eraser is latex-free, which provides enough toughness for heavy erasures. These smudge-free erasers wipe the page clean, making corrections easy and fast.\n&#11088; WIDE APPLICATION. Suitable for all general writing applications such as notes, letters, and you can use it for office, classroom, home and other occasions.",BAZIC Products,12-count,us
2,B07G7F6JZ6,"Emraw Pre Sharpened Round Primary Size No 2 Jumbo Pencils for Preschoolers, Elementary Kids - Pack of 8 Premium Fat Pencils","<p><b>Emraw Pre-Sharpened #2 HB Wood Pencils - Pack of 8 Pre-Schoolers Pencils</b></p><p>Looking for pencils for kid, office or every day general purpose repetitive use? Tired of pencils that do not sharpen well? Disappointed having broken leads repeatedly? Or, looking for pencils that require less sharpening and lasts longer?</p><p>Search no further, Emraw num 2 jumbo pencils pack have you covered. You just found the best deal. You will love them. Just the right size for pre-kindergarten toddlers. Perfect for elderly hands having difficulty grasping smaller items. Each pencil is pre-sharpened so you can literally pick straight from the box and start using. HB (no2) hardness grading with thicker leads make them perfect for most types of writing or sketching.</p><p>Made of high quality non-toxic cedar wood so they sharpen nicely with any quality dual-holed hand sharpener or adjustable electric sharpener, (not compatible with standard pencil sharpener). Rounded circular sh

In [9]:
for locale, _df in prod_cat.groupby(['product_locale']):
    print(locale, _df.product_id.is_unique)
    assert _df.product_id.is_unique

es True
jp True
us True


In [10]:
def get_product_input(row):
    product_columns = ['product_title', 'product_locale', 'product_description', 'product_bullet_point']
    product_info = ''
    for col in product_columns:
        product_info += str(row.get(col, '')) + ' '
    return product_info.strip()

d = {'product_title': 'hi hello', 'product_locale': 'en', 'product_description': 'nike', 'product_bullet_point': 'shoes'}
assert get_product_input(d) == 'hi hello en nike shoes'

In [11]:
prod_cat.fillna('', inplace=True)
prod_cat['product_input'] = prod_cat.apply(lambda row: get_product_input(row), axis=1)

In [12]:
prod_cat.head(4)

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,product_input
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box",,"144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpening\nStrong medium-soft lead produces long-lasting, smooth, readable strokes\nRounded hexagonal shape with satin-smooth finish for a secure, comfortable grip\nSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes",Amazon Basics,Yellow,us,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box us 144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpening\nStrong medium-soft lead produces long-lasting, smooth, readable strokes\nRounded hexagonal shape with satin-smooth finish for a secure, comfortable grip\nSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes"
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack","<p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developing new exciting products for our consumers.</p> <p><strong>FOCUSED ON OUR MISSION</strong></p><p>Our currency is ideas. We thrive on imagination, passion and leadership. We have great products and will to continue to rise with our customer expectations.</p> <p><strong>SUCCESS BASED ON SATISFACTION</strong></p><p>Each and every product we send out, we expect our 100% customer satisfaction. While our supplies are sourced from all across the world, our success stems from individual consumer fulfillment. We create products that people want to recommend to others.</p>","&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each pack comes with 12 un-sharpened pencils. Unlike most others out there, these allow kids the simple joy of sharpening a brand new pencil for the very 1st time!\n&#11088; CLASSIC BARREL. Made from premium-quality wood pencil is bonded with a break-resistant number two core for smooth writing as easily erased. Hexagonal shape prevents rolling and comfortable to hold.\n&#11088; QUALITY. Pencils are #2 HB for general writing, drawing and sketching. Pencils glide smoothly over paper and is highly break resistant. Pencils include a smudge-resistant eraser top for convenience.\n&#11088; LATEX-FREE ERASER. The smudge-less eraser is latex-free, which provides enough toughness for heavy erasures. These smudge-free erasers wipe the page clean, making corrections easy and fast.\n&#11088; WIDE APPLICATION. Suitable for all general writing applications such as notes, letters, and you can use it for office, classroom, home and other occasions.",BAZIC Products,12-count,us,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack us <p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and dev

In [13]:
escapes = ''.join([chr(char) for char in range(1, 32)])
translator = str.maketrans('', '', escapes)
prod_cat['product_input'] = prod_cat['product_input'].apply(lambda inp: inp.translate(translator))

In [14]:
prod_cat.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,product_input
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box",,"144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpening\nStrong medium-soft lead produces long-lasting, smooth, readable strokes\nRounded hexagonal shape with satin-smooth finish for a secure, comfortable grip\nSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes",Amazon Basics,Yellow,us,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box us 144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpeningStrong medium-soft lead produces long-lasting, smooth, readable strokesRounded hexagonal shape with satin-smooth finish for a secure, comfortable gripSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes"
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack","<p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developing new exciting products for our consumers.</p> <p><strong>FOCUSED ON OUR MISSION</strong></p><p>Our currency is ideas. We thrive on imagination, passion and leadership. We have great products and will to continue to rise with our customer expectations.</p> <p><strong>SUCCESS BASED ON SATISFACTION</strong></p><p>Each and every product we send out, we expect our 100% customer satisfaction. While our supplies are sourced from all across the world, our success stems from individual consumer fulfillment. We create products that people want to recommend to others.</p>","&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each pack comes with 12 un-sharpened pencils. Unlike most others out there, these allow kids the simple joy of sharpening a brand new pencil for the very 1st time!\n&#11088; CLASSIC BARREL. Made from premium-quality wood pencil is bonded with a break-resistant number two core for smooth writing as easily erased. Hexagonal shape prevents rolling and comfortable to hold.\n&#11088; QUALITY. Pencils are #2 HB for general writing, drawing and sketching. Pencils glide smoothly over paper and is highly break resistant. Pencils include a smudge-resistant eraser top for convenience.\n&#11088; LATEX-FREE ERASER. The smudge-less eraser is latex-free, which provides enough toughness for heavy erasures. These smudge-free erasers wipe the page clean, making corrections easy and fast.\n&#11088; WIDE APPLICATION. Suitable for all general writing applications such as notes, letters, and you can use it for office, classroom, home and other occasions.",BAZIC Products,12-count,us,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack us <p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developin

In [15]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>')
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

prod_cat['product_input'] = prod_cat['product_input'].apply(lambda inp: cleanhtml(inp))

In [16]:
prod_cat.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,product_input
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box",,"144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpening\nStrong medium-soft lead produces long-lasting, smooth, readable strokes\nRounded hexagonal shape with satin-smooth finish for a secure, comfortable grip\nSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes",Amazon Basics,Yellow,us,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box us 144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpeningStrong medium-soft lead produces long-lasting, smooth, readable strokesRounded hexagonal shape with satin-smooth finish for a secure, comfortable gripSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes"
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack","<p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developing new exciting products for our consumers.</p> <p><strong>FOCUSED ON OUR MISSION</strong></p><p>Our currency is ideas. We thrive on imagination, passion and leadership. We have great products and will to continue to rise with our customer expectations.</p> <p><strong>SUCCESS BASED ON SATISFACTION</strong></p><p>Each and every product we send out, we expect our 100% customer satisfaction. While our supplies are sourced from all across the world, our success stems from individual consumer fulfillment. We create products that people want to recommend to others.</p>","&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each pack comes with 12 un-sharpened pencils. Unlike most others out there, these allow kids the simple joy of sharpening a brand new pencil for the very 1st time!\n&#11088; CLASSIC BARREL. Made from premium-quality wood pencil is bonded with a break-resistant number two core for smooth writing as easily erased. Hexagonal shape prevents rolling and comfortable to hold.\n&#11088; QUALITY. Pencils are #2 HB for general writing, drawing and sketching. Pencils glide smoothly over paper and is highly break resistant. Pencils include a smudge-resistant eraser top for convenience.\n&#11088; LATEX-FREE ERASER. The smudge-less eraser is latex-free, which provides enough toughness for heavy erasures. These smudge-free erasers wipe the page clean, making corrections easy and fast.\n&#11088; WIDE APPLICATION. Suitable for all general writing applications such as notes, letters, and you can use it for office, classroom, home and other occasions.",BAZIC Products,12-count,us,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack us BACK TO BAZICOur goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect. COMMITTED TO VALUESWe are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developing new exciting products for our consumers. FOCUSED ON OUR 

In [17]:
from sklearn.model_selection import train_test_split
train = []
test = []
for local, _df in prod_cat.groupby(['product_locale']):
    train_df, test_df = train_test_split(_df, test_size=0.3, random_state=9)
    train.append(train_df)
    test.append(test_df)

In [18]:
train_df = pd.concat(train, ignore_index=True)
test_df = pd.concat(test, ignore_index=True)

In [19]:
train_df.shape, test_df.shape

((618706, 8), (265162, 8))

In [20]:
train_samples = []
test_samples = []
train_df.apply(lambda row: train_samples.append(InputExample(texts=[str(row['product_input']), str(row['product_input'])])), axis=1)
test_df.apply(lambda row: test_samples.append(InputExample(texts=[str(row['product_input']), str(row['product_input'])], label=1.0)), axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
265157    None
265158    None
265159    None
265160    None
265161    None
Length: 265162, dtype: object

In [21]:
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=8, name='test-simcse')

In [22]:
len(train_samples), len(test_samples)

(618706, 265162)

In [ ]:
import torch
# Configure the training
train_batch_size = 4
num_epochs = 3

# Use MultipleNegativesRankingLoss for SimCSE
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)
model_save_path = './'

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

logging.info("Performance before training")
dev_evaluator(model)

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=500,
          warmup_steps=warmup_steps,
          output_path=model_save_path,
          save_best_model=True,
          use_amp=torch.cuda.is_available(),
          )

In [ ]:
try:
    model.save('/kaggle/working/')
except:
    print("Trainer save stopped")